# 1. pepbind 파이프라인 디버그 노트북

이 노트북은 한 번 돌린 pepbind 파이프라인 결과(ColabFold, Vina, PLIP, PRODIGY)를
단계별로 확인하기 위한 디버그용입니다.

- 맨 위 `WS_ROOT` 경로만 **현재 워크스페이스 폴더**로 바꿔서 사용하세요.  
  예) `/home/aisys/work/pipeline/PDP_20251122_150359`
- 각 셀을 위에서부터 차례대로 실행하면 됩니다.

# 2. 공통 설정 + WS_ROOT

In [2]:
from pathlib import Path
import os, json, re
import pandas as pd

# ▼ 여기만 현재 워크스페이스(PDP_날짜_시간 폴더)로 수정해서 사용하세요.
WS_ROOT = Path("/home/aisys/work/pipeline/PDP_20251122_174430").expanduser()

print("WS_ROOT:", WS_ROOT)
print("하위 폴더 목록:")
for p in WS_ROOT.iterdir():
    print(" -", p.name)


WS_ROOT: /home/aisys/work/pipeline/PDP_20251122_174430
하위 폴더 목록:
 - fasta
 - results
 - temp
 - pdb


# 3. 주요 서브폴더 구조 확인

In [3]:
colabfold_out = WS_ROOT / "pdb" / "colabfold_output"
results_dir   = WS_ROOT / "results"
vina_dir      = results_dir / "vina"
plip_dir      = results_dir / "plip"
prodigy_dir   = results_dir / "prodigy"

print("ColabFold out:", colabfold_out)
print("Vina dir     :", vina_dir)
print("PLIP dir     :", plip_dir)
print("PRODIGY dir  :", prodigy_dir)

for d in [colabfold_out, vina_dir, plip_dir, prodigy_dir]:
    print("\n[", d, "]")
    if d.exists():
        for p in sorted(d.iterdir()):
            print(" -", p.name)
    else:
        print(" (존재하지 않음)")


ColabFold out: /home/aisys/work/pipeline/PDP_20251122_174430/pdb/colabfold_output
Vina dir     : /home/aisys/work/pipeline/PDP_20251122_174430/results/vina
PLIP dir     : /home/aisys/work/pipeline/PDP_20251122_174430/results/plip
PRODIGY dir  : /home/aisys/work/pipeline/PDP_20251122_174430/results/prodigy

[ /home/aisys/work/pipeline/PDP_20251122_174430/pdb/colabfold_output ]
 - cite.bibtex
 - colabfold_batch.log
 - complex_0.a3m
 - complex_0.done.txt
 - complex_0_coverage.png
 - complex_0_env
 - complex_0_pae.png
 - complex_0_pairgreedy
 - complex_0_plddt.png
 - complex_0_predicted_aligned_error_v1.json
 - complex_0_scores_rank_001_alphafold2_multimer_v3_model_1_seed_000.json
 - complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_1.a3m
 - complex_1.done.txt
 - complex_1_coverage.png
 - complex_1_env
 - complex_1_pae.png
 - complex_1_pairgreedy
 - complex_1_plddt.png
 - complex_1_predicted_aligned_error_v1.json
 - complex_1_scores_rank_001_alphafold2_mul

# 4. ColabFold rank_001 PDB + ipTM 확인

In [4]:
# ColabFold rank_001 PDB 목록 확인
rank1_pdbs = sorted(colabfold_out.glob("*rank_001*.*pdb"))
print(f"rank_001 PDB 개수: {len(rank1_pdbs)}")
for p in rank1_pdbs:
    print(" -", p.name)

# ipTM 추출 함수 (파이프라인과 동일 로직)
def load_iptm_scores(colabfold_out_dir, rank1_pdbs):
    iptms = {}
    if not colabfold_out_dir.exists():
        return iptms

    for pdb in rank1_pdbs:
        base = pdb.stem
        prefix = base.split("_unrelaxed")[0]
        found_val = None

        # scores*.json 후보
        candidates = list(colabfold_out_dir.glob(f"{base}*scores*.json"))
        if not candidates:
            candidates = list(colabfold_out_dir.glob(f"{prefix}*scores*.json"))

        for js in candidates:
            try:
                with open(js) as f:
                    data = json.load(f)
            except Exception:
                continue

            if isinstance(data, dict):
                v = data.get("iptm")
                if isinstance(v, (int, float)):
                    found_val = float(v)
                    break
                v = data.get("iptm+ptm")
                if isinstance(v, (int, float)):
                    found_val = float(v)
                    break

        # ranking_debug 후보
        if found_val is None:
            rd_candidates = [
                colabfold_out_dir / f"{base}_ranking_debug.json",
                colabfold_out_dir / f"{prefix}_ranking_debug.json",
                colabfold_out_dir / "ranking_debug.json",
            ]
            for rd in rd_candidates:
                if not rd.exists():
                    continue
                try:
                    with open(rd) as f:
                        data = json.load(f)
                except Exception:
                    continue

                if isinstance(data, dict):
                    v = data.get("iptm") or data.get("iptm+ptm")
                    if isinstance(v, (int, float)):
                        found_val = float(v)
                        break

        if found_val is not None:
            iptms[base] = found_val

    return iptms

iptm_vals = load_iptm_scores(colabfold_out, rank1_pdbs)
print("\n[ipTM 값]")
for k, v in iptm_vals.items():
    print(k, "->", v)

rank_001 PDB 개수: 10
 - complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb

[ipTM 값]
complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000 -> 0.19
complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000 -> 0.06
complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_mode

# 5. Vina summary / debug

In [5]:
# Vina summary / debug 확인
vina_summary_path = vina_dir / "vina_summary.csv"
if vina_summary_path.exists():
    vina_df = pd.read_csv(vina_summary_path)
    print("Vina summary:")
    display(vina_df)
else:
    print("vina_summary.csv 가 없습니다:", vina_summary_path)

vina_debug_path = vina_dir / "vina_debug.txt"
if vina_debug_path.exists():
    print("\n[Vina debug 로그]")
    print(vina_debug_path.read_text())
else:
    print("\nVina debug 로그 파일이 없습니다:", vina_debug_path)

Vina summary:


,complex,vina_score,vina_status,receptor_pdbqt,ligand_pdbqt,log_file
0,complex_0_unrelaxed_rank_001_alphafold2_multim...,-4.3000,정상,complex_0_unrelaxed_rank_001_alphafold2_multim...,complex_0_unrelaxed_rank_001_alphafold2_multim...,complex_0_unrelaxed_rank_001_alphafold2_multim...
1,complex_1_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),complex_1_unrelaxed_rank_001_alphafold2_multim...,complex_1_unrelaxed_rank_001_alphafold2_multim...,complex_1_unrelaxed_rank_001_alphafold2_multim...
2,complex_2_unrelaxed_rank_001_alphafold2_multim...,-5.1530,정상,complex_2_unrelaxed_rank_001_alphafold2_multim...,complex_2_unrelaxed_rank_001_alphafold2_multim...,complex_2_unrelaxed_rank_001_alphafold2_multim...
3,complex_3_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),complex_3_unrelaxed_rank_001_alphafold2_multim...,complex_3_unrelaxed_rank_001_alphafold2_multim...,complex_3_unrelaxed_rank_001_alphafold2_multim...
4,complex_4_unrelaxed_rank_001_alphafold2_multim...,-2.5410,정상,complex_4_unrelaxed_rank_001_alphafold2_multim...,complex_4_unrelaxed_rank_001_alphafold2_multim...,complex_4_unrelaxed_rank_001_alphafold2_multim...
5,complex_5_unrelaxed_rank_001_alphafold2_multim...,-3.8800,정상,complex_5_unrelaxed_rank_001_alphafold2_multim...,complex_5_unrelaxed_rank_001_alphafold2_multim...,complex_5_unrelaxed_rank_001_alphafold2_multim...
6,complex_6_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),complex_6_unrelaxed_rank_001_alphafold2_multim...,complex_6_unrelaxed_rank_001_alphafold2_multim...,complex_6_unrelaxed_rank_001_alphafold2_multim...
7,complex_7_unrelaxed_rank_001_alphafold2_multim...,NaN,실패: Vina 실행 에러(code=1) (PDBQT parsing error: f...,complex_7_unrelaxed_rank_001_alphafold2_multim...,complex_7_unrelaxed_rank_001_alphafold2_multim...,complex_7_unrelaxed_rank_001_alphafold2_multim...
8,complex_8_unrelaxed_rank_001_alphafold2_multim...,-0.1207,정상,complex_8_unrelaxed_rank_001_alphafold2_multim...,complex_8_unrelaxed_rank_001_alphafold2_multim...,complex_8_unrelaxed_rank_001_alphafold2_multim...
9,complex_9_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),complex_9_unrelaxed_rank_001_alphafold2_multim...,complex_9_unrelaxed_rank_001_alphafold2_multim...,complex_9_unrelaxed_rank_001_alphafold2_multim...



[Vina debug 로그]
complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	-4.3
complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상(점수=0.0)	0.0
complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	-5.153
complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상(점수=0.0)	0.0
complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	-2.541
complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	-3.88
complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상(점수=0.0)	0.0
complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	실패: Vina 실행 에러(code=1) (PDBQT parsing error: flex residue/ligand 태그 문제)	log=complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_vina_stdout.txt
complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	-0.1207
complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상(점수=0.0)	0.0


# 6. 특정 complex의 Vina stdout 자세히 보기

In [12]:
# 디테일하게 보고 싶은 complex ID 지정 (vina_summary.csv 의 'complex' 값 중 하나)
target_complex = "complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000"

complex_dir = vina_dir / target_complex
print("complex_dir:", complex_dir)

if complex_dir.exists():
    print("\n[Vina complex 폴더 파일 목록]")
    for p in sorted(complex_dir.iterdir()):
        print(" -", p.name)
else:
    print("폴더가 없습니다. target_complex 이름을 확인하세요.")

stdout_path = complex_dir / f"{target_complex}_vina_stdout.txt"
if stdout_path.exists():
    print("\n[Vina stdout 내용 (앞 2000자)]")
    txt = stdout_path.read_text()
    print(txt[:2000])
else:
    print("\nVina stdout 파일이 없습니다:", stdout_path)

complex_dir: /home/aisys/work/pipeline/PDP_20251122_174430/results/vina/complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000

[Vina complex 폴더 파일 목록]
 - complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdb
 - complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdbqt
 - complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb
 - complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt
 - complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_vina_stdout.txt

[Vina stdout 내용 (앞 2000자)]
=== STDOUT ===
AutoDock Vina 23d1252-mod
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #

# 7. PLIP summary / debug

In [7]:
# PLIP 요약 및 디버그 로그
plip_summary_path = plip_dir / "plip_summary.csv"
if plip_summary_path.exists():
    plip_df = pd.read_csv(plip_summary_path)
    print("PLIP summary:")
    display(plip_df)
else:
    print("plip_summary.csv 가 없습니다:", plip_summary_path)

plip_run_debug = plip_dir / "plip_run_debug.txt"
if plip_run_debug.exists():
    print("\n[PLIP run debug]")
    print(plip_run_debug.read_text())
else:
    print("\nplip_run_debug.txt 가 없습니다:", plip_run_debug)

plip_parse_debug = plip_dir / "plip_parse_debug.txt"
if plip_parse_debug.exists():
    print("\n[PLIP parse debug]")
    print(plip_parse_debug.read_text())
else:
    print("\nplip_parse_debug.txt 가 없습니다:", plip_parse_debug)

PLIP summary:


,complex,plip_total_interactions,plip_hbond,plip_hydrophobic,plip_saltbridge,plip_status
0,complex_3_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
1,complex_9_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
2,complex_8_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
3,complex_7_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
4,complex_1_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
5,complex_4_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
6,complex_2_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
7,complex_0_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
8,complex_6_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...
9,complex_5_unrelaxed_rank_001_alphafold2_multim...,NaN,NaN,NaN,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...



[PLIP run debug]
complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	OK	log=complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_plip.log
complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	OK	log=complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_plip.log
complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	OK	log=complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_plip.log
complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	OK	log=complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_plip.log
complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	OK	log=complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_plip.log
complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	OK	log=complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_plip.log
complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_

# 8. 특정 complex PLIP 폴더 보기

In [8]:
target_complex = "complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000"
complex_plip_dir = plip_dir / target_complex
print("complex_plip_dir:", complex_plip_dir)

if complex_plip_dir.exists():
    print("\n[PLIP complex 폴더 파일 목록]")
    for p in sorted(complex_plip_dir.iterdir()):
        print(" -", p.name)
else:
    print("폴더가 없습니다. target_complex 이름을 확인하세요.")

complex_plip_dir: /home/aisys/work/pipeline/PDP_20251122_174430/results/plip/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000

[PLIP complex 폴더 파일 목록]
 - complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_plip.log
 - complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_protonated.pdb


# 9. PRODIGY summary / debug

In [9]:
# PRODIGY 요약 및 디버그
prodigy_summary_path = prodigy_dir / "prodigy_summary.csv"
if prodigy_summary_path.exists():
    prodigy_df = pd.read_csv(prodigy_summary_path)
    print("PRODIGY summary:")
    display(prodigy_df)
else:
    print("prodigy_summary.csv 가 없습니다:", prodigy_summary_path)

prodigy_debug_path = prodigy_dir / "prodigy_debug.txt"
if prodigy_debug_path.exists():
    print("\n[PRODIGY debug]")
    print(prodigy_debug_path.read_text())
else:
    print("\nprodigy_debug.txt 가 없습니다:", prodigy_debug_path)

PRODIGY summary:


,complex,PRODIGY_dG,prodigy_status
0,complex_0_unrelaxed_rank_001_alphafold2_multim...,0.0,정상
1,complex_1_unrelaxed_rank_001_alphafold2_multim...,NaN,파싱실패: stdout에서 ΔG 패턴 없음
2,complex_2_unrelaxed_rank_001_alphafold2_multim...,2.0,정상
3,complex_3_unrelaxed_rank_001_alphafold2_multim...,NaN,파싱실패: stdout에서 ΔG 패턴 없음
4,complex_4_unrelaxed_rank_001_alphafold2_multim...,NaN,파싱실패: stdout에서 ΔG 패턴 없음
5,complex_5_unrelaxed_rank_001_alphafold2_multim...,2.0,정상
6,complex_6_unrelaxed_rank_001_alphafold2_multim...,NaN,파싱실패: stdout에서 ΔG 패턴 없음
7,complex_7_unrelaxed_rank_001_alphafold2_multim...,NaN,파싱실패: stdout에서 ΔG 패턴 없음
8,complex_8_unrelaxed_rank_001_alphafold2_multim...,NaN,파싱실패: stdout에서 ΔG 패턴 없음
9,complex_9_unrelaxed_rank_001_alphafold2_multim...,NaN,파싱실패: stdout에서 ΔG 패턴 없음



[PRODIGY debug]
complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	0.0
complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	파싱실패: stdout에서 ΔG 패턴 없음	None
complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	2.0
complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	파싱실패: stdout에서 ΔG 패턴 없음	None
complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	파싱실패: stdout에서 ΔG 패턴 없음	None
complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	정상	2.0
complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	파싱실패: stdout에서 ΔG 패턴 없음	None
complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	파싱실패: stdout에서 ΔG 패턴 없음	None
complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	파싱실패: stdout에서 ΔG 패턴 없음	None
complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000	파싱실패: stdout에서 ΔG 패턴 없음	None


# 10. 특정 complex PRODIGY stdout/stderr 보기

In [10]:
target_complex = "complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000"

out_txt = prodigy_dir / f"{target_complex}_prodigy.txt"
err_txt = prodigy_dir / f"{target_complex}_prodigy.stderr.txt"

print("stdout 파일:", out_txt)
print("stderr 파일:", err_txt)

if out_txt.exists():
    print("\n[PRODIGY stdout 내용]")
    print(out_txt.read_text())
else:
    print("\nstdout 파일이 없습니다.")

if err_txt.exists():
    print("\n[PRODIGY stderr 내용]")
    print(err_txt.read_text())
else:
    print("\nstderr 파일이 없습니다.")

stdout 파일: /home/aisys/work/pipeline/PDP_20251122_174430/results/prodigy/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_prodigy.txt
stderr 파일: /home/aisys/work/pipeline/PDP_20251122_174430/results/prodigy/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_prodigy.stderr.txt

[PRODIGY stdout 내용]
[!] Structure contains gaps:
	A ALA1 < Fragment 0 > A THR222
	B GLU1 < Fragment 1 > B ARG2
	B VAL3 < Fragment 2 > B LEU4

[+] Executing 1 task(s) in total
##########################################
[+] Processing structure complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_model0
[+] No. of intermolecular contacts: 18
[+] No. of charged-charged contacts: 2.0
[+] No. of charged-polar contacts: 0.0
[+] No. of charged-apolar contacts: 13.0
[+] No. of polar-polar contacts: 0.0
[+] No. of apolar-polar contacts: 0.0
[+] No. of apolar-apolar contacts: 3.0
[+] Percentage of apolar NIS residues: 36.17
[+] Percentage of charged NIS residues: 29.79


# 11. 통합 테이블(선택)

In [11]:
# 선택: Vina / PRODIGY / PLIP / ipTM 를 한 테이블로 대략 합치기
dfs = []

if 'vina_df' in globals():
    dfs.append(vina_df[['complex', 'vina_score', 'vina_status']])
if 'prodigy_df' in globals():
    dfs.append(prodigy_df[['complex', 'PRODIGY_dG', 'prodigy_status']])
if 'plip_df' in globals():
    dfs.append(plip_df[['complex', 'plip_total_interactions', 'plip_status']])

from functools import reduce

if dfs:
    merged = reduce(lambda left, right: pd.merge(left, right, on='complex', how='outer'), dfs)
    # ipTM 추가
    iptm_series = pd.Series(iptm_vals, name='ipTM')
    iptm_df = iptm_series.reset_index()
    iptm_df.columns = ['complex', 'ipTM']

    merged = merged.merge(iptm_df, on='complex', how='left')

    print("통합 테이블:")
    display(merged)
else:
    print("결합할 데이터프레임이 없습니다.")

통합 테이블:


,complex,vina_score,vina_status,PRODIGY_dG,prodigy_status,plip_total_interactions,plip_status,ipTM
0,complex_0_unrelaxed_rank_001_alphafold2_multim...,-4.3000,정상,0.0,정상,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.19
1,complex_1_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),NaN,파싱실패: stdout에서 ΔG 패턴 없음,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.06
2,complex_2_unrelaxed_rank_001_alphafold2_multim...,-5.1530,정상,2.0,정상,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.17
3,complex_3_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),NaN,파싱실패: stdout에서 ΔG 패턴 없음,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.05
4,complex_4_unrelaxed_rank_001_alphafold2_multim...,-2.5410,정상,NaN,파싱실패: stdout에서 ΔG 패턴 없음,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.05
5,complex_5_unrelaxed_rank_001_alphafold2_multim...,-3.8800,정상,2.0,정상,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.15
6,complex_6_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),NaN,파싱실패: stdout에서 ΔG 패턴 없음,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.05
7,complex_7_unrelaxed_rank_001_alphafold2_multim...,NaN,실패: Vina 실행 에러(code=1) (PDBQT parsing error: f...,NaN,파싱실패: stdout에서 ΔG 패턴 없음,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.05
8,complex_8_unrelaxed_rank_001_alphafold2_multim...,-0.1207,정상,NaN,파싱실패: stdout에서 ΔG 패턴 없음,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.07
9,complex_9_unrelaxed_rank_001_alphafold2_multim...,0.0000,정상(점수=0.0),NaN,파싱실패: stdout에서 ΔG 패턴 없음,NaN,실패: report.xml/report.txt 없음 (PLIP 출력에 상호작용 요약...,0.05
